In [3]:
import requests

In [11]:
requests.get(url="http://0.0.0.0:8001/v1/performance").text

'["Hello performance"]'

In [13]:
requests.get(url="http://0.0.0.0:8001/v1/aderencia").text

'["Hello aderencia"]'